# Problem 1 - Working with RDDs (5 points)

This is an interactive PySpark session. Remember that when you open this notebook the `SparkContext` and `SparkSession` are already created, and they are in the `sc` and `spark` variables, respectively. You can run the following two cells to make sure that the Kernel is active.

**Do not insert any additional cells than the ones that are provided.**

In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [2]:
spark

In the following cell, make an RDD called `top1m` that contains the contents of the file `top-1m.csv` that you placed into the cluster's HDFS.

In [13]:
top1m = sc.textFile("/user/hadoop/top-1m.csv")

There is one element in the RDD for each line in the file. The `.count()` method will compute how many lines are in the file. In the following cell, type the expression to count the lines in the `top1m` RDD. Run the cell and see the result.

In [18]:
top1m.count()

1000000

## Count the `.com` domains

How many of the websites in this RDD are in the .com domain?

In the following cell, write a code snippet that finds the records with `.com` and counts them. (Hint: use a regular expression.)

In [71]:
import re
com_re = re.compile("\.com$")

def extract(line):
    m = com_re.search(line)
    if m:
        return "number of records with .com"
    else:
        return "number of records without .com"
    
dotcom = top1m.map( lambda line: [ extract( line ), 1 ])

dotcom.countByKey()

defaultdict(int,
            {'number of records with .com': 490422,
             'number of records without .com': 509578})

## Histogram the Top Level Domains (TLDs)

What is the distribution of TLDs in the top 1 million websites? We can quickly compute this using the RDD function `countByValue()`.

In the following cell, write a function called `tld` (in Python) that takes a domain name string and outputs the top-level domain.

In [104]:
def tld(name):
    return(name.split(".")[-1])

In the following cell, map the `top1m` RDD using `tld` into a new RDD called `tlds`. 

In [113]:
tlds = top1m.map( lambda line: tld(line))

In the following two cells, evaluate `top1m.first()` and  `tlds.first()` to see if the first line of `top1m` transformed by `tld` is properly represented as the first line of `tlds`. 

In [111]:
top1m.first()

u'1,youtube.com'

In [114]:
tlds.first()

u'com'

Look at the first 50 elements of `top1m` by evaluating `top1m.take(50)`.

In [115]:
top1m.take(50)

[u'1,youtube.com',
 u'2,google.com',
 u'3,facebook.com',
 u'4,baidu.com',
 u'5,wikipedia.org',
 u'6,reddit.com',
 u'7,yahoo.com',
 u'8,google.co.in',
 u'9,qq.com',
 u'10,taobao.com',
 u'11,sohu.com',
 u'12,tmall.com',
 u'13,amazon.com',
 u'14,twitter.com',
 u'15,vk.com',
 u'16,instagram.com',
 u'17,sina.com.cn',
 u'18,google.co.jp',
 u'19,live.com',
 u'20,jd.com',
 u'21,360.cn',
 u'22,weibo.com',
 u'23,yandex.ru',
 u'24,google.co.uk',
 u'25,google.com.br',
 u'26,netflix.com',
 u'27,google.de',
 u'28,twitch.tv',
 u'29,google.ru',
 u'30,google.fr',
 u'31,login.tmall.com',
 u'32,pornhub.com',
 u'33,alipay.com',
 u'34,xvideos.com',
 u'35,google.com.hk',
 u'36,google.it',
 u'37,google.es',
 u'38,google.com.mx',
 u'39,ebay.com',
 u'40,bing.com',
 u'41,yahoo.co.jp',
 u'42,google.ca',
 u'43,t.co',
 u'44,hao123.com',
 u'45,ok.ru',
 u'46,microsoft.com',
 u'47,pages.tmall.com',
 u'48,wikia.com',
 u'49,mail.ru',
 u'50,imgur.com']

Try the same thing with the `tlds` RDD to make sure that the first 50 lines were properly transformed.


In [116]:
tlds.take(50)

[u'com',
 u'com',
 u'com',
 u'com',
 u'org',
 u'com',
 u'com',
 u'in',
 u'com',
 u'com',
 u'com',
 u'com',
 u'com',
 u'com',
 u'com',
 u'com',
 u'cn',
 u'jp',
 u'com',
 u'com',
 u'cn',
 u'com',
 u'ru',
 u'uk',
 u'br',
 u'com',
 u'de',
 u'tv',
 u'ru',
 u'fr',
 u'com',
 u'com',
 u'com',
 u'com',
 u'hk',
 u'it',
 u'es',
 u'mx',
 u'com',
 u'com',
 u'jp',
 u'ca',
 u'co',
 u'com',
 u'ru',
 u'com',
 u'com',
 u'com',
 u'ru',
 u'com']

At this point, `tlds.countByValue()` would give us a list of each TLD and the number of times that it appears in the top1m file. Note that this function returns the results as a `defaultDict` in the Python environemnt, not as an RDD. But we want it reverse sorted by count. To do this, we can set a variable called `tlds_and_counts` equal to `tlds.countByValue()` and then reverse the order, sort, and take the top 50, like this:

```
tlds_and_counts = tlds.countByValue()
counts_and_tlds = [(count,domain) for (domain,count) in tlds_and_counts.items()]
```

In the following cell, run the code above to produce the Python Dictionary.

In [117]:
tlds_and_counts = tlds.countByValue()
counts_and_tlds = [(count,domain) for (domain,count) in tlds_and_counts.items()]

In the following cell, reverse sort `counts_and_tlds` and display the first 50.

In [126]:
counts_and_tlds.sort(reverse=True)
counts_and_tlds[:50]

[(490422, u'com'),
 (48554, u'org'),
 (45130, u'ru'),
 (42911, u'net'),
 (28239, u'de'),
 (20067, u'br'),
 (17312, u'uk'),
 (15470, u'ir'),
 (13488, u'pl'),
 (12789, u'in'),
 (11318, u'au'),
 (10655, u'fr'),
 (10649, u'it'),
 (9000, u'info'),
 (8018, u'es'),
 (6946, u'nl'),
 (6811, u'jp'),
 (6810, u'gr'),
 (6738, u'ca'),
 (6524, u'cz'),
 (6503, u'mx'),
 (6490, u'co'),
 (6236, u'ua'),
 (5239, u'tw'),
 (5230, u'se'),
 (5064, u'io'),
 (4743, u'eu'),
 (4365, u'cn'),
 (3831, u'hu'),
 (3806, u'me'),
 (3711, u'tv'),
 (3710, u'dk'),
 (3536, u'za'),
 (3486, u'ch'),
 (3479, u'sk'),
 (3187, u'us'),
 (3179, u'ar'),
 (3152, u'kr'),
 (3110, u'ro'),
 (3090, u'id'),
 (3069, u'vn'),
 (2920, u'no'),
 (2832, u'edu'),
 (2783, u'at'),
 (2776, u'cl'),
 (2759, u'be'),
 (2569, u'tr'),
 (2325, u'biz'),
 (2117, u'pt'),
 (1979, u'xyz')]

**Question:** `top1m.collect()[0:50]` and `top1m.take(50)` produce the same result. Which one is more efficient and why? Put your answer in the cell below.

The functoin call "top1m.take(50)" is more efficient. Because .collect() implies gathering and .take() does not. The function .take() means to accept. When the collect operation is used on an RDD, the dataset is being copied to the driver, in our case is the master node. A memory exception will be trown if the the collected data is too large to fit in memory. Take function can be used to get capped number of elements instead. 

When you finish this problem, click on the File -> 'Save and Checkpoint' in the menu bar to make sure that the latest version of the workbook file is saved. Also, before you close this notebook and move on, make sure you disconnect your SparkContext, otherwise you will not be able to re-allocate resources. Remember, you will commit the .ipynb file to the repository for submission (in the master node terminal.)

In [ ]:
sc.stop()